# Local Flood Mapping

In the context of this notebook we referred to "local flood mapping" as performing the computation on you own machine, as opposed to remote processing as explained in the next notebook.

We will begin by loading the `flood` module from the `dask_flood_mapper`. You could use the default Dask settings, but it is often good to fine tune te settings to you machine's specifications. We can do this by setting the Dask scheduler through the `Client` of `dask.distributed`.

In [1]:
from dask_flood_mapper import flood
from dask.distributed import Client

Here we set the Dask Scheduler with the `Client`, where we avoid inter-worker communication which is common for working with `numpy` and `xarray` in this case.

In [ ]:
client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="20GB")
client

/home/mschobbe/.local/share/virtualenvs/dask-flood-mapper-z0eq0g_n/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43137 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://128.131.72.130:43137/status,
Dashboard: http://128.131.72.130:43137/status,Workers: 1
Total threads: 2,Total memory: 18.63 GiB
Status: running,Using processes: False
Comm: inproc://128.131.72.130/17266/12,Workers: 1
Dashboard: http://128.131.72.130:43137/status,Total threads: 2
Started: Just now,Total memory: 18.63 GiB
Comm: inproc://128.131.72.130/17266/15,Total threads: 2
Dashboard: http://128.131.72.130:34879/status,Memory: 18.63 GiB
Nanny: None,


/home/mschobbe/.local/share/virtualenvs/dask-flood-mapper-z0eq0g_n/lib/python3.12/site-packages/dask/_task_spec.py:740: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)
/home/mschobbe/.local/share/virtualenvs/dask-flood-mapper-z0eq0g_n/lib/python3.12/site-packages/dask/_task_spec.py:741: RuntimeWarning: divide by zero encountered in divide
  return self.func(*new_argspec)


No we are ready to map a flood. As an example we use here storm Babet which hit the Danish and Northern coast of Germany at the 20<sup>th</sup> of October 2023 [Wikipedia](https://en.wikipedia.org/wiki/Storm_Babet). We target an area around Zingst at the Baltic coast of Northern Germany.

In [3]:
time_range = "2022-10-11/2022-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
fd = flood.decision(bbox=bbox, datetime=time_range).compute()
fd

sigma naught datacube processed
harmonic parameter datacube processed
projected local incidence angle processed


<xarray.DataArray 'decision' (time: 8, y: 1048, x: 2793)> Size: 187MB
array([[[nan, nan, nan, ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        [ 0.,  0.,  0., ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        [ 0.,  0.,  0., ...,  0., nan, nan],
        ...,
        [nan, nan,  0., ...,  0.,  0.,  0.],
        [nan, nan,  0., ...,  0., nan, nan],
        [nan, nan,  0., ..., nan, nan, nan]]])
Coordinates:
  * x            (x) float64 22kB 12.3 12.3 12.3 12.3 ... 13.1 13.1 13.1 13.1
  * y            (y) float64 8kB 54.6 54.6 54.6 54.6 ... 54.3 54.3 54.3 54.3
  * time         (time) datetime64[ns] 64B 2022-10-11T05:25:01 ... 2022-10-23...
    spatial_ref  int64 8B 0
Attributes:
    _FillValue:  nan

Since `flood.decision` does not make a distinction between Sentinel-1 observations over land or over sea, we need to mask pixels over water. For the example here we will load a mask distributed along with this package. But in general this step is left to the users own discretion.

In [4]:
from importlib.resources import files
import xarray as xr

data_text = files("dask_flood_mapper.data").joinpath("wcover.tif")
wcover = xr.open_dataarray(data_text)

Now we are ready to view our results. We use here `hvplot` to create an animation of the flood's extent over time.

In [5]:
from bokeh.models import FixedTicker
import hvplot.xarray

fd = fd.where(wcover != 80)
colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
fd.hvplot.quadmesh(
    x="x",
    y="y",
    rasterize=True,
    geo=True,
    cmap=["rgba(0, 0, 1, 0.1)", "darkred"],
).opts(frame_height=400, colorbar_opts={**colorbar_opts})

BokehModel(combine_events=True, render_bundle={'docs_json': {'724d29ba-0231-4602-97bc-aebd887b5717': {'version…

In [6]:
client.close()